In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd drive/MyDrive/Colab\ Notebooks/dec-pt

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/dec-pt'
/content/drive/MyDrive/Colab Notebooks/dec-pt


In [ ]:
import sys
sys.path.append("..")
from models.stacked_dae import LitStackedDenoisingAE
from utils.utils import set_dropout
from utils.LitDataLoader import TracesDataset
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import optuna
from optuna.integration import PyTorchLightningPruningCallback

In [ ]:
def objective(trial):
    n_layers = trial.suggest_int("n_layers", 2, 4)
    n_layers = 3
    dropout_dae = trial.suggest_discrete_uniform("dropout_dae", 0.0, 0.5, 0.1)
    dropout_sdae = trial.suggest_discrete_uniform("dropout_sdae", 0.0, 0.5, 0.1)
    input_dim = 1725
    layers = []
    
    layer_dim = int(trial.suggest_int("n_units_l1", 500, 2500, 200))
    layers.append(layer_dim)

    layer_dim = int(trial.suggest_int("n_units_l2", 400, 1500, 100))
    layers.append(layer_dim)

    layer_dim = int(trial.suggest_int("n_units_l3", 400, 2000, 100))
    layers.append(layer_dim)

    lr_dae = trial.suggest_loguniform('lr_dae', 1e-5, 1)
    lr_sdae = trial.suggest_loguniform('lr_sdae', 1e-5, 1)
    optim = trial.suggest_categorical('optimizer', ['SGD', 'Adam', 'AdamW', 'RMSprop'])
    latent_dim = trial.suggest_int('latent_dim', 4, 30)
    corrupt = trial.suggest_uniform('corrupt', 0.0, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])

    data_loader = TracesDataset(data_dir='data/Haleh/', mode='fit', batch_size=batch_size)
    data_loader.prepare_data()
    data_loader.setup()
    train_loader = data_loader.train_dataloader()
    val_loader = data_loader.val_dataloader()
    lit_sdae = LitStackedDenoisingAE(input_dim=input_dim, hidden_dims=layers, dropout=dropout_sdae,
                                     latent_dim=latent_dim, learning_rate=lr_sdae,
                                     corruption_type='gaussian', corrupt=corrupt, optimizer=optim)
    print(lit_sdae)
    lit_sdae.pretrain(train_loader, val_loader, trial, dropout=dropout_dae, max_epochs=40, batch_size=batch_size, lr=lr_dae)
    if dropout_sdae != dropout_dae:
        set_dropout(lit_sdae, drop_rate=dropout_sdae)
    logger = TensorBoardLogger('tb_logs', 'SDAE_Tune')
    trainer = pl.Trainer(gpus=1,
                         max_epochs=70,
                         auto_lr_find=True,
                         logger=logger,
                         callbacks=[PyTorchLightningPruningCallback(trial, monitor='val_loss')]
                         )
    trainer.fit(lit_sdae, train_loader, val_loader)

In [ ]:
study = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study.optimize(objective, n_trials=40)
print('Best trial: score {}, params {}'.format(study.best_trial.value, study.best_trial.params))
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_intermediate_values(study)

[I 2021-05-12 10:01:30,124] A new study created in memory with name: no-name-eeccfb6e-4195-4a25-8426-bd41a1a787ef
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


cuda
LitStackedDenoisingAE(
  (encoder): Sequential(
    (0): Sequential(
      (linear): Linear(in_features=1725, out_features=2100, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): Sequential(
      (linear): Linear(in_features=2100, out_features=800, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (2): Sequential(
      (linear): Linear(in_features=800, out_features=1400, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (3): Sequential(
      (linear): Linear(in_features=1400, out_features=29, bias=True)
    )
  )
  (decoder): Sequential(
    (0): Sequential(
      (linear): Linear(in_features=29, out_features=1400, bias=True)
      (activation): ReLU()
    )
    (1): Sequential(
      (linear): Linear(in_features=1400, out_features=800, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (2): Seque


  | Name           | Type    | Params
-------------------------------------------
0 | enc_activation | ReLU    | 0     
1 | dropout        | Dropout | 0     
-------------------------------------------
7.2 M     Trainable params
0         Non-trainable params
7.2 M     Total params
28.995    Total estimated model params size (MB)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type    | Params
-------------------------------------------
0 | enc_activation | ReLU    | 0     
1 | dec_activation | ReLU    | 0     
2 | dropout        | Dropout | 0     
-------------------------------------------
3.4 M     Trainable params
0         Non-trainable params
3.4 M     Total params
13.452    Total estimated model params size (MB)


...


cuda
LitStackedDenoisingAE(
  (encoder): Sequential(
    (0): Sequential(
      (linear): Linear(in_features=1725, out_features=2500, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (1): Sequential(
      (linear): Linear(in_features=2500, out_features=1100, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (2): Sequential(
      (linear): Linear(in_features=1100, out_features=1900, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (3): Sequential(
      (linear): Linear(in_features=1900, out_features=21, bias=True)
    )
  )
  (decoder): Sequential(
    (0): Sequential(
      (linear): Linear(in_features=21, out_features=1900, bias=True)
      (activation): ReLU()
    )
    (1): Sequential(
      (linear): Linear(in_features=1900, out_features=1100, bias=True)
      (activation): ReLU()
      (dropout): Dropout(p=0.5, inplace=False)
    )
    (2): S

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type    | Params
-------------------------------------------
0 | enc_activation | ReLU    | 0     
1 | dropout        | Dropout | 0     
-------------------------------------------
8.6 M     Trainable params
0         Non-trainable params
8.6 M     Total params
34.517    Total estimated model params size (MB)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type    | Params
-------------------------------------------
0 | enc_activation | ReLU    | 0     
1 | dec_activation | ReLU    | 0     
2 | dropout        | Dropout | 0     
-------------------------------------------
5.5 M     Trainable params
0         Non-trainable params
5.5 M     Total params
22.014    Total estimated model params size (MB)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type    | Params
-------------------------------------------
0 | enc_activation | ReLU    | 0     
1 | dec_activation | ReLU    | 0     
2 | dropout        | Dropout | 0     
-------------------------------------------
4.2 M     Trainable params
0         Non-trainable params
4.2 M     Total params
16.732    Total estimated model params size (MB)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type    | Params
-------------------------------------------
0 | enc_activation | ReLU    | 0     
1 | dec_activation | ReLU    | 0     
2 | dropout        | Dropout | 0     
-------------------------------------------
5.5 M     Trainable params
0         Non-trainable params
5.5 M     Total params
22.014    Total estimated model params size (MB)
